# Imports

In [1]:
from pathlib import Path
import os
import sys
sys.path.append(str(Path(os.getcwd()).parent / 'middleware/middleware'))

In [2]:
import pandas as pd
from glob import glob
from functools import reduce
import random
import re
from src.pipelines.alpha import Alpha
from src.helpers import correct_encoding, read_jsonl, write_jsonl
from IPython.display import clear_output

# Loading export

In [26]:
df = pd.read_csv('data/exports/export_CR_PETCT-01012016_an.csv', sep='\t', encoding='utf-8')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41183 entries, 0 to 41182
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NUMERO_EXAMEN   41183 non-null  object
 1   TYPE_EXAMEN     41183 non-null  object
 2   LIBELLE_EXAMEN  41183 non-null  object
 3   NUM_SALLE       41183 non-null  object
 4   DATE_EXAMEN     41183 non-null  object
 5   HEURE_EXAMEN    41183 non-null  object
 6   COMPTE_RENDU    41183 non-null  object
 7   year            41183 non-null  int64 
 8   month           41183 non-null  int64 
 9   cr_an           41183 non-null  object
dtypes: int64(2), object(8)
memory usage: 3.1+ MB


,NUMERO_EXAMEN,TYPE_EXAMEN,LIBELLE_EXAMEN,NUM_SALLE,DATE_EXAMEN,HEURE_EXAMEN,COMPTE_RENDU,year,month,cr_an
0,A10836454533,MN,TEP CORPS ENTIER,MBPET1,04/01/16,08:01,Installation répertoriée sous le n° M540008 Au...,16,1,Installation répertoriée sous le n° M540008 Au...
1,A10887608918,MN,TEP CORPS ENTIER,MBPET1,04/01/16,14:52,Installation répertoriée sous le n° M540008 Au...,16,1,Installation répertoriée sous le n° M540008 Au...
2,A10890051978,MN,TEP CORPS ENTIER,MBPET1,04/01/16,14:44,Installation répertoriée sous le n° M540008 Au...,16,1,Installation répertoriée sous le n° M540008 Au...
3,A10883861577,MN,TEP CORPS ENTIER,MBPET1,04/01/16,11:11,Installation répertoriée sous le n° M540008 Au...,16,1,Installation répertoriée sous le n° M540008 Au...
4,A10893853897,MN,TEP CORPS ENTIER,MBPET1,04/01/16,15:29,Installation répertoriée sous le n° M540008 Au...,16,1,Installation répertoriée sous le n° M540008 Au...


# Loading of existing samples

In [4]:
existing_samples = reduce(lambda a, b: a+b, [read_jsonl(path) for path in glob('data/suv/dones/*.jsonl')])
existing_samples = [es['text'].strip() for es in existing_samples]
existing_samples[:5]

["Hypermétabolisme intense de la région vulvaire très suspect de récidive locale (SUVmax à 9,5 pour une référence hépatique de l'ordre de 3)",
 "Image nodulaire de la loge thymique 15x7x12 mm, aux contours réguliers, discrètement fixante (SUVmax de 2.3)  non retrouvée rétrospectivement sur l'examen  TEP précédent, priori le plus vraisemblablement ganglionnaire",
 'Adénopathies hypermétaboliques lombo-aortiques, iliaques primitives, internes et externes bilatérales avec une cible métabolique en iliaque externe droite (SUVmax à 4.5) sur une lésion de 26x10.5mm dans le plan axial.',
 "Epaississement circonférentiel pleural droit et épanchement hypermétabolique (SUVmax à 7,3 pour une référence hépatique à 2,8), de façon diffuse mais hétérogène avec présence de plusieurs nodules hypermétaboliques disséminés sur l'hémichamp pulmonaire droit.\r\nAtélectasie lobaire inférieure droite non hypermétabolique.\r\nAdénopathies hypermétaboliques pré-trachéale droite, sous-carinaire et hilaire droite,

# Loading model

In [6]:
nlp = Alpha('../middleware/middleware/models').nlp

# Making sample

In [11]:
seed = 0
N = 100
MATCHER = re.compile(r'suv', re.I)
SECTION = (3, 'obs')
RESULT_PATH = 'data/suv/to_annotated/suv_demo.jsonl'

new_samples = []
random_ids = list(range(len(df)))
random.seed(seed)
random.shuffle(random_ids)

while len(new_samples) < N and len(random_ids) != 0:
    cr = df.loc[random_ids.pop(), 'cr_an']
    if isinstance(cr, float):
        continue
    try:
        doc = nlp(cr)
    except:
        pass
    
    if len(doc._.sections) < SECTION[0] or doc._.sections[SECTION[0]].label_ != SECTION[1]:
        continue
        
    for sent in list(doc._.sections[3].as_doc().sents):
        txt = sent.text
        if MATCHER.search(txt) != None and txt.strip() not in existing_samples and txt.strip() not in new_samples:
            new_samples.append(txt)
            
    clear_output(wait=True)
    print(f'{len(new_samples)}/{N}, {len(new_samples)/N:.2f}')
    
write_jsonl(RESULT_PATH, [{'text':txt} for txt in new_samples])

102/100, 1.02


# Gathering samples

In [12]:
samples = reduce(lambda a, b: a+b, [read_jsonl(path) for path in glob('data/suv/dones/*.jsonl')])
len(samples)

3001

In [13]:
write_jsonl('data/suv/suv_loc.jsonl', samples)

# Explore samples

# Make samples

In [27]:
df = df.drop(['year', 'month', 'cr_an'], axis=1)
df.sort_index(ascending=False)
df.loc[0:199].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NUMERO_EXAMEN   200 non-null    object
 1   TYPE_EXAMEN     200 non-null    object
 2   LIBELLE_EXAMEN  200 non-null    object
 3   NUM_SALLE       200 non-null    object
 4   DATE_EXAMEN     200 non-null    object
 5   HEURE_EXAMEN    200 non-null    object
 6   COMPTE_RENDU    200 non-null    object
dtypes: object(7)
memory usage: 11.1+ KB


In [28]:
df.loc[0:199].to_csv('data/devia/devia_0_199.csv', index=False, encoding='utf-8')
df.loc[200:399].to_csv('data/devia/devia_200_399.csv', index=False, encoding='utf-8')
df.loc[400:599].to_csv('data/devia/devia_400_599.csv', index=False, encoding='utf-8')
df.loc[600:799].to_csv('data/devia/devia_600_799.csv', index=False, encoding='utf-8')
df.loc[800:].to_csv('data/devia/devia_800_999.csv', index=False, encoding='utf-8')